In [13]:
import sys
COMP_NAME = "icecube-neutrinos-in-deep-ice"
sys.path.append(f"/home/anjum/kaggle/{COMP_NAME}/")

import pandas as pd
import numpy as np
import torch

from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from graphnet.models.gnn import DynEdge
from graphnet.models.graph_builders import KNNGraphBuilder

from src.config import INPUT_PATH, OUTPUT_PATH

In [39]:
batch_id = 1
event_id = 24

sensors = pd.read_csv(INPUT_PATH / "sensor_geometry_v2.csv")
meta = pd.read_parquet(INPUT_PATH / "train_meta.parquet").query(f"batch_id == {batch_id}")
batch = pd.read_parquet(INPUT_PATH / "train" / f"batch_{batch_id}.parquet")

In [40]:
meta.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117


In [41]:
batch.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True


In [42]:
sensors.head()

,sensor_id,x,y,z,string,qe
0,0,-256.14,-521.08,496.03,0,1.0
1,1,-256.14,-521.08,479.01,0,1.0
2,2,-256.14,-521.08,461.99,0,1.0
3,3,-256.14,-521.08,444.97,0,1.0
4,4,-256.14,-521.08,427.95,0,1.0


In [43]:
event = pd.merge(batch.loc[event_id], sensors, on="sensor_id")
event

,sensor_id,time,charge,auxiliary,x,y,z,string,qe
0,3918,5928,1.325,True,303.41,335.64,206.58,65,1.0
1,4157,6115,1.175,True,-145.45,374.24,212.73,69,1.0
2,3520,6492,0.925,True,505.27,257.88,-174.60,58,1.0
3,3520,14523,1.325,True,505.27,257.88,-174.60,58,1.0
4,5041,6665,0.225,True,-9.68,-79.50,181.00,84,1.0
...,...,...,...,...,...,...,...,...,...
56,3452,17812,0.725,True,382.35,238.90,-46.12,57,1.0
57,48,18053,0.975,True,-256.14,-521.08,-320.97,0,1.0
58,3267,18095,1.125,True,11.87,179.19,42.78,54,1.0
59,3267,18102,1.425,True,11.87,179.19,42.78,54,1.0


In [44]:
def preprocessing(event):
    event["x"] /= 500
    event["y"] /= 500
    event["z"] /= 500
    event["time"] = (event["time"] - 1.0e04) / 3.0e4
    event["charge"] = np.log10(event["charge"]) / 3.0
    event["auxiliary"] = event["auxiliary"].astype(int)
    
    return event[["x", "y", "z", "time", "charge", "qe", "auxiliary"]]


event = preprocessing(event)
event

,x,y,z,time,charge,qe,auxiliary
0,0.60682,0.67128,0.41316,-0.135733,0.040739,1.0,1
1,-0.29090,0.74848,0.42546,-0.129500,0.023346,1.0,1
2,1.01054,0.51576,-0.34920,-0.116933,-0.011286,1.0,1
3,1.01054,0.51576,-0.34920,0.150767,0.040739,1.0,1
4,-0.01936,-0.15900,0.36200,-0.111167,-0.215939,1.0,1
...,...,...,...,...,...,...,...
56,0.76470,0.47780,-0.09224,0.260400,-0.046554,1.0,1
57,-0.51228,-1.04216,-0.64194,0.268433,-0.003665,1.0,1
58,0.02374,0.35838,0.08556,0.269833,0.017051,1.0,1
59,0.02374,0.35838,0.08556,0.270067,0.051272,1.0,1


In [45]:
event.describe()

,x,y,z,time,charge,qe,auxiliary
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,0.023824,-0.058633,0.052818,0.092286,-0.020030,1.034426,0.786885
std,0.610632,0.520048,0.502226,0.113998,0.078332,0.105096,0.412907
min,-1.053260,-1.042160,-0.876160,-0.135733,-0.252321,1.000000,0.000000
25%,-0.469900,-0.535040,-0.410940,0.021400,-0.046554,1.000000,1.000000
50%,0.023740,-0.031200,0.204360,0.086900,-0.011286,1.000000,1.000000
75%,0.317880,0.358380,0.424260,0.187800,0.029379,1.000000,1.000000
max,1.152740,0.980440,0.999020,0.301033,0.199779,1.350000,1.000000


In [46]:
x = torch.tensor(event.values, dtype=torch.float32)
x.shape, x.dtype

(torch.Size([61, 7]), torch.float32)

In [56]:
target = meta.query(f"batch_id == {batch_id} & event_id == {event_id}")
y = torch.tensor(target[["azimuth", "zenith"]].values, dtype=torch.float32)
y

tensor([[5.0296, 2.0875]])

In [57]:
data = Data(x=x, y=y)
data.n_pulses = torch.tensor(x.shape[0], dtype=torch.int32)
data

Data(x=[61, 7], y=[1, 2], n_pulses=61)

In [48]:
builder = KNNGraphBuilder(nb_nearest_neighbours=8)

data_out = builder(data)
data_out

Data(x=[61, 7], n_pulses=61, edge_index=[2, 488])

In [49]:
# In final version use a PTGeo dataset & dataloader
batch = Batch.from_data_list([data_out, data_out])
batch

DataBatch(x=[122, 7], n_pulses=[2], edge_index=[2, 976], batch=[122], ptr=[3])

In [50]:
model = DynEdge(nb_inputs=7, global_pooling_schemes=["min", "max", "mean", "sum"])

In [51]:
out = model(batch)
out.shape

torch.Size([2, 128])

In [52]:
torch.save(data, "test.pt")

In [53]:
test = torch.load("test.pt")
test

Data(x=[61, 7], n_pulses=61, edge_index=[2, 488])